# Dependências

In [21]:
#!pip install numba==0.49.1
#!pip install numba==0.55.0
#!pip install numba
#!pip uninstall numba -y
#!pip install llvmlite

In [22]:
#!pip install llvmlite --ignore-installed

In [23]:
#!pip install gensim==3.6.0
#!pip install binary gensim==3.6.0

In [24]:
#!pip3 uninstall numpy -y

In [25]:
#!pip install numpy

# Bibliotecas

In [2]:
import pandas as pd
import numpy as np
import psycopg2 as postgres
import psycopg2.extras
import math
import csv
import random

from gensim.models import Word2Vec
from tqdm import tqdm
from sshtunnel import SSHTunnelForwarder
from tqdm import tqdm

import numba
from numba import njit
#from numba import cuda

/home/salatiel/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


# Funções

## Utilidades

In [3]:
def executeQuery(conn, sql):
    record = None
    try:
        #print(sql)
        cur = conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
        cur.execute(sql)
        record = cur.fetchall()
        cur.close()
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()
    return record

In [4]:
def executeInsert(conn, sql):
    sucess = False
            
    try:
        #print(sql)
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
        cur.close()
        sucess = True
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()

    return sucess

In [5]:
#Recebe um id e retorna as categorias e checkin do lugar
def getPOIInformation(conn, business_id):
    
    sql = """
        SELECT id, checkin_count, name, level FROM pois_information WHERE id  = \'"""+str(business_id)+ """\'
    ;"""

    result = executeQuery(conn, sql)

    return result

In [6]:
#Recebe um id e retorna as categorias e checkin do lugar
def getZoningInformation(conn, object_id):
    
    sql = """
        SELECT id FROM zoning_1618_pois WHERE objectid  = \'"""+str(object_id)+ """\'
    ;"""

    result = executeQuery(conn, sql)

    return result

In [7]:
#Recebe um id e retorna as categorias e checkin do lugar
def getZones(conn, object_id):
    
    sql = """
        SELECT objectid FROM zoning_1618_pois
    ;"""

    result = executeQuery(conn, sql)

    return result

In [8]:
#Encontra todos os pontos do bin centrado em um POI trazendo suas informações (categorias e checkin)
def getBinPOIsInformation(conn, business_id, bin_number):

    result = None

    sql = """
        SELECT fk_poi_id_context as id, name, level, checkin_count, distance_m 
        FROM bins_pois_information 
        WHERE fk_poi_id_center = \'"""+str(business_id)+"""\' AND fk_bin_number = """+str(bin_number)+""" order by distance_m asc;"""

    result = executeQuery(conn, sql)

    return result

In [9]:
def getPOIsDistance(conn, center_poi_id, context_poi_id):
    
    result = None

    sql = """
        SELECT distance_m
        FROM has_context_poi 
        WHERE fk_poi_id_center = \'"""+str(center_poi_id)+"""\' AND fk_poi_id_context = """+str(context_poi_id)+""";"""

    result = executeQuery(conn, sql)

    return result

In [10]:
#Recebe um id e retorna as categorias e checkin do lugar
def getAllPOIInformation(conn):
    
    sql = """
        SELECT id as business_id, latitude, longitude, categories, checkin_count FROM poi;"""

    result = executeQuery(conn, sql)

    return result

In [11]:
#LENTO
def calculate_L(POI_path, df):
    dst = 0
    segment = []
    for poi_id in POI_path:
        #print(poi_id)
        segment.append(poi_id)
        if (len(segment) > 2):
            segment.pop(0)

        if (len(segment) == 2):

            p_idx = POI_path[-2]
            p_idx = df.query("business_id=='"+str(p_idx)+"'").index[0]
            n_idx = POI_path[-1]
            n_idx = df.query("business_id=='"+str(n_idx)+"'").index[0]
        
            dst += distances_matrix[p_idx][n_idx]
            
    return dst

In [12]:
#Meio LENTO
def calculate_L_(POI_path, df):
    dst = 0
    
    for i in range(len(POI_path)-1):

        p_idx = POI_path[i]
        p_idx = df.query("business_id=='"+str(p_idx)+"'").index[0]
        n_idx = POI_path[i+1]
        n_idx = df.query("business_id=='"+str(n_idx)+"'").index[0]

        dst += distances_matrix[p_idx][n_idx]
            
    return dst

In [13]:
#Mais Rápido
def calculate_Li_(POI_path, df, i, L):

    p_idx = POI_path[i-1]
    p_idx = df.query("business_id=='"+str(p_idx)+"'").index[0]

    c_idx = POI_path[i]
    c_idx = df.query("business_id=='"+str(c_idx)+"'").index[0]

    n_idx = POI_path[i+1]
    n_idx = df.query("business_id=='"+str(n_idx)+"'").index[0]

    #Retirando antigo segmento
    segment_distance = L - distances_matrix[p_idx][n_idx]
    
    #Tamanho do novo segmento
    dst = segment_distance + distances_matrix[p_idx][c_idx] + distances_matrix[c_idx][n_idx]
            
    return dst

In [14]:
#Essa versão não utilizada a otimização do Numba
def generate_segment(W, L, POI_path):
    
    final_path = [pt for pt in POI_path]

    for wp in W:

        shortest_path = np.inf
        optimal_list = None

        #Testar WP em todas as posições entre 1 e len(list)-2
        for j in range(1, len(final_path)):
            
            local_list = [fp for fp in final_path]
            
            #Testar WP em todas as posições entre 1 e len(list)-2
            local_list.insert(j, wp)
            

            p_idx = local_list[j-1] 
            c_idx = local_list[j]
            n_idx = local_list[j+1]

            #Retirando antigo segmento
            segment_distance = L - distances_matrix[p_idx][n_idx]

            #Tamanho do novo segmento
            dst = segment_distance + distances_matrix[p_idx][c_idx] + distances_matrix[c_idx][n_idx]

            if (dst < shortest_path):
                shortest_path = dst
                optimal_list = [ll for ll in local_list] 
            
        final_path = [ol for ol in optimal_list] 
        L = shortest_path
    
    return final_path

In [15]:
# function optimized to run on gpu 
#@jit(target_backend='cuda')
#@numba.njit
#from numba.typed import List

@njit
#@cuda.jit
def generate_segment_numba(W, L, POI_path):
    
    final_path = [pt for pt in POI_path]

    for wp in W:

        shortest_path = np.inf
        optimal_list = None

        #Testar WP em todas as posições entre 1 e len(list)-2
        for j in range(1, len(final_path)):
            
            local_list = [fp for fp in final_path]
            
            #Testar WP em todas as posições entre 1 e len(list)-2
            local_list.insert(j, wp)
            

            p_idx = local_list[j-1] 
            c_idx = local_list[j]
            n_idx = local_list[j+1]

            #Retirando antigo segmento
            segment_distance = L - distances_matrix[p_idx][n_idx]

            #Tamanho do novo segmento
            dst = segment_distance + distances_matrix[p_idx][c_idx] + distances_matrix[c_idx][n_idx]

            if (dst < shortest_path):
                shortest_path = dst
                optimal_list = [ll for ll in local_list] 
            
        final_path = [ol for ol in optimal_list] 
        L = shortest_path
    
    return final_path

In [16]:
from timeit import default_timer as timer    

# normal function to run on cpu 
def func(a):                                 
    for i in range(100000): 
        a[i]+= 1      

# function optimized to run on gpu  
#@jit(nopython=True)
#@cuda.jit
#(target ="cuda")    
@njit
def func2(a): 
    for j in range(100000):
        for i in range(100000): 
            a[i]+= 1
#if __name__=="__main__": 

n = 100000                           
a = np.ones(n, dtype = np.float64) 
#b = np.ones(n, dtype = np.float32) 

start = timer() 
func(a) 
print("without GPU:", timer()-start)     

start = timer() 
func2(a) 
print("with GPU:", timer()-start) 

without GPU: 0.0163154059991939
with GPU: 4.272350682000251


# Document Corpus

In [17]:
#Essa versão analisa os POIs em um contexto, encontra os dois mais distantes e monta o caminho mais curto entre eles
def Shortest_Path_FT(object_id, df):
    
    #print('processing POI:', poi_id)
    POI_path_bi = []
    POI_path_fl = []
    
    #print('processing:', object_id)
    
    try:
    
        with SSHTunnelForwarder(
        ('150.165.15.171', 23456),
        #ssh_private_key="</path/to/private/ssh/key>",
        ### in my case, I used a password instead of a private key
        ssh_username="salatiel",
        ssh_password="Salatiel@Lacina123", 
        remote_bind_address=('localhost', 5432)) as server:

            server.start()
            #print("server connected")

            params = {'database': 'boston',
                   'user': 'salatiel',
                   'password': 'root',
                   'host': 'localhost',
                   'port': server.local_bind_port
            }

            connection = psycopg2.connect(**params)


            #Obtendo informações de categorias e checkin do poi central
            #[business_id, checkin, category]
            poi_information = getZoningInformation(connection, object_id)


            #Executa o processo se há dados
            #Um caminho preciso do no mínimo dois POIs
            if (len(poi_information) >= 2):

                #Restringindo as possibilidades de tipos
                columns = list(dict(poi_information[0]).keys())
                poi_information = pd.DataFrame(poi_information, columns = columns)

                
                #Shortest Path construction
                pois_idx = []
                for i, cp_idx in poi_information.iterrows():
                    pois_idx.append(df.query("business_id=='"+str(cp_idx['id'])+"'").index[0])

                #Encontrar Ps e Pe
                max_distances = []
                for poi_idx_01 in pois_idx:
                    for poi_idx_02 in pois_idx:
                        if poi_idx_01 != poi_idx_02:
                            max_distances.append((poi_idx_01, poi_idx_02, distances_matrix[poi_idx_01][poi_idx_02]))

                furthest_pair = max_distances[0]
                for md in max_distances:
                    if md[2] > furthest_pair[2]:
                        furthest_pair = md

                        
                POI_path = [furthest_pair[0], furthest_pair[1]]
                L = furthest_pair[2]

                #A is all POIs
                W = set(pois_idx).difference(set(POI_path))
                
                #Ajuste das variáveis para a função generate_segment

                W = list(W)

                #Adicionar o POI de A que tem o menor tamanho de caminho em L e atualizar W
                if (len(W) > 0):

                    #Não otimizado
                    #POI_path_fl = generate_segment(W, L, POI_path)
                    
                    #Otimizado pelo numba
                    POI_path_fl = generate_segment_numba(W, L, POI_path)
                else:

                    POI_path_fl = POI_path.copy()
                
                POI_path_bi = [df.iloc[pp]['business_id'] for pp in POI_path_fl]
                
            connection.close()
    
    except Exception as e:
        print(str(e))         
        
    return POI_path_bi

In [36]:
#TESTE LOCAL

object_id = "12269"
df = poi_df

#print('processing POI:', poi_id)
POI_path_bi = []
POI_path_fl = []

#print('processing:', object_id)

try:

    with SSHTunnelForwarder(
    ('150.165.15.171', 23456),
    #ssh_private_key="</path/to/private/ssh/key>",
    ### in my case, I used a password instead of a private key
    ssh_username="salatiel",
    ssh_password="Salatiel@Lacina123", 
    remote_bind_address=('localhost', 5432)) as server:

        server.start()
        #print("server connected")

        params = {'database': 'boston',
               'user': 'salatiel',
               'password': 'root',
               'host': 'localhost',
               'port': server.local_bind_port
        }

        connection = psycopg2.connect(**params)


        #Obtendo informações de categorias e checkin do poi central
        #[business_id, checkin, category]
        poi_information = getZoningInformation(connection, object_id)


        #Executa o processo se há dados
        #Um caminho preciso do no mínimo dois POIs
        if (len(poi_information) >= 2):

            #Restringindo as possibilidades de tipos
            columns = list(dict(poi_information[0]).keys())
            poi_information = pd.DataFrame(poi_information, columns = columns)


            #Shortest Path construction
            pois_idx = []
            for i, cp_idx in poi_information.iterrows():
                pois_idx.append(df.query("business_id=='"+str(cp_idx['id'])+"'").index[0])

            #Encontrar Ps e Pe
            max_distances = []
            for poi_idx_01 in pois_idx:
                for poi_idx_02 in pois_idx:
                    if poi_idx_01 != poi_idx_02:
                        max_distances.append((poi_idx_01, poi_idx_02, distances_matrix[poi_idx_01][poi_idx_02]))

            furthest_pair = max_distances[0]
            for md in max_distances:
                if md[2] > furthest_pair[2]:
                    furthest_pair = md


            POI_path = [furthest_pair[0], furthest_pair[1]]
            L = furthest_pair[2]

            #A is all POIs
            W = set(pois_idx).difference(set(POI_path))

            #Ajuste das variáveis para a função generate_segment

            W = list(W)

            #Adicionar o POI de A que tem o menor tamanho de caminho em L e atualizar W
            if (len(W) > 0):

                #Não otimizado
                #POI_path_fl = generate_segment(W, L, POI_path)

                #Otimizado pelo numba
                POI_path_fl = generate_segment_numba(W, L, POI_path)
            else:

                POI_path_fl = POI_path.copy()

            POI_path_bi = [df.iloc[pp]['business_id'] for pp in POI_path_fl]

        connection.close()

except Exception as e:
    print(str(e))         

In [37]:
#RESULTADO
POI_path_bi

['Iq2Ey5AiDK9JUKddyDLZ7g',
 '3ejv6lfK3Qd9Q-pb_9YVUA',
 '7dzyLlNQXwuPsmvquuaVIA',
 'S3nk3mllSk7QI-sRqG92Gg',
 '3JOdRJS4PFkpfnNVxQtcmQ',
 '41PtFgRt3DQa4J4FHDZkRA',
 'dcXeYPp3UAgiCzGzmEz6aQ',
 'h7skkHYh4JXZfARD6QhRBg',
 'WO10z5s4b6bg8G97emPn7A',
 'dZ800YCAc-HnvXInE7GU9g',
 'KR0O7xKEm6rgE711HkkXTA',
 'Choq0lds-DnnH0bUqx8FHQ',
 'ehbByKSGZ28AHXlqUCVzIA',
 'fBsbGcUFnMO_7yXfikNjGg',
 'iHdg7BupnCVc3FYlrzxTrg',
 'KYSd1eCFQY0uCON1GXmAhQ',
 'ILAaAXr4Varupm8MUSuwrA',
 '3B1z5h8Y39hnEXbRc9jTHA',
 'qjAgVbVXuECartJqPeXErQ',
 'u3f-W5yNwkSKJ8LjFtYCxg',
 'k7W_o3LpZaxMPWphXGS1MA',
 'WwNticOnmmkHrMRyAGR4rw',
 'H8_9t4Tlw2bcKW9qdqAM8w',
 'lBtiaHCwi-QHAs_ReWClhw',
 'AAZFsMBLNgNg58eA06bFJA',
 'yXgJ3_DPc4SQ-MDIw5Fkvw',
 '-n_qDyqJ6oq1yQb1qc0hsQ',
 'PIWXl9Bjw2Kvj6TUkIacsA']

In [38]:
#TIPO DE DADO DO POI_INFORMATION
poi_information

,id
0,-n_qDyqJ6oq1yQb1qc0hsQ
1,iHdg7BupnCVc3FYlrzxTrg
2,7dzyLlNQXwuPsmvquuaVIA
3,Iq2Ey5AiDK9JUKddyDLZ7g
4,3ejv6lfK3Qd9Q-pb_9YVUA
5,k7W_o3LpZaxMPWphXGS1MA
6,Choq0lds-DnnH0bUqx8FHQ
7,H8_9t4Tlw2bcKW9qdqAM8w
8,41PtFgRt3DQa4J4FHDZkRA
9,ehbByKSGZ28AHXlqUCVzIA


In [39]:
#DATAFRAME DE POI
poi_df = poi_df[['business_id', 'latitude', 'longitude']]
poi_df.head()

,business_id,latitude,longitude
0,hcRxdDg7DYryCxCoI8ySQA,42.338544,-71.106842
1,jGennaZUr2MsJyRhijNBfA,42.363442,-71.025781
2,iPD8BBvea6YldQZPHzVrSQ,42.342673,-71.084239
3,WuoNSCjFBH4OQTSNGc4l6w,42.350004,-71.081931
4,zHMsnpMPanzRx-A6AwGSdw,42.352135,-71.063102


# Carregando dados de POIs

In [20]:
with SSHTunnelForwarder(
    ('150.165.15.171', 23456),
    #ssh_private_key="</path/to/private/ssh/key>",
    ### in my case, I used a password instead of a private key
    ssh_username="salatiel",
    ssh_password="Salatiel@Lacina123",
    remote_bind_address=('localhost', 5432)) as server:

        server.start()
        #print("server connected")

        params = {'database': 'boston',
               'user': 'salatiel',
               'password': 'root',
               'host': 'localhost',
               'port': server.local_bind_port
        }

        connection = psycopg2.connect(**params)

        #Buscar informações dos POIs
        poi_df = getAllPOIInformation(connection)
        if (len(poi_df) > 0):
            tags = list(dict(poi_df[0]).keys())
            poi_df = pd.DataFrame(poi_df, columns = tags)

poi_df = poi_df.dropna()
poi_df['latitude'] = poi_df['latitude'].astype(float)
poi_df['longitude'] = poi_df['longitude'].astype(float)
poi_df.head()

,business_id,latitude,longitude,categories,checkin_count
0,hcRxdDg7DYryCxCoI8ySQA,42.338544,-71.106842,"Restaurants, Shopping, Shopping Centers",24
1,jGennaZUr2MsJyRhijNBfA,42.363442,-71.025781,"Sandwiches, Food, Restaurants, Breakfast & Bru...",856
2,iPD8BBvea6YldQZPHzVrSQ,42.342673,-71.084239,"Creperies, Restaurants, Food, Coffee & Tea, Br...",7
3,WuoNSCjFBH4OQTSNGc4l6w,42.350004,-71.081931,"Beauty & Spas, Hair Salons, Active Life, Day Spas",138
4,zHMsnpMPanzRx-A6AwGSdw,42.352135,-71.063102,"Day Spas, Tanning, Hair Stylists, Massage, Spr...",42


In [24]:
from scipy.spatial.distance import cdist
distances_matrix = cdist(poi_df[['latitude', 'longitude']].values, poi_df[['latitude', 'longitude']].values)
distances_matrix[0]

array([0.        , 0.08479843, 0.02297677, ..., 0.03257799, 0.03288391,
       0.05160452])

# Carregando dados de Zonas

In [25]:
#Carregando ZONAS
with SSHTunnelForwarder(
    ('150.165.15.171', 23456),
    #ssh_private_key="</path/to/private/ssh/key>",
    ### in my case, I used a password instead of a private key
    ssh_username="salatiel",
    ssh_password="Salatiel@Lacina123",
    remote_bind_address=('localhost', 5432)) as server:

        server.start()
        #print("server connected")

        params = {'database': 'boston',
               'user': 'salatiel',
               'password': 'root',
               'host': 'localhost',
               'port': server.local_bind_port
        }

        connection = psycopg2.connect(**params)

        #Buscar informações dos POIs
        query = "SELECT distinct(objectid) FROM zoning_1618_pois"
        result = executeQuery(connection, query)
        
        final_zoning_gdf = pd.DataFrame(result)

In [26]:
final_zoning_gdf.head()

,objectid
0,12530
1,12175
2,11963
3,11638
4,12269


In [27]:
obj_id_list = list(final_zoning_gdf['objectid'])
obj_id_list[:5]

[12530, 12175, 11963, 11638, 12269]

In [23]:
results = []
for obj_id in tqdm(obj_id_list):
    results.append(Shortest_Path_FT(obj_id, poi_df))

100%|██████████| 598/598 [05:41<00:00,  1.75it/s]


In [30]:
Shortest_Path_FT("12269", poi_df)

['Iq2Ey5AiDK9JUKddyDLZ7g',
 '3ejv6lfK3Qd9Q-pb_9YVUA',
 '7dzyLlNQXwuPsmvquuaVIA',
 'S3nk3mllSk7QI-sRqG92Gg',
 '3JOdRJS4PFkpfnNVxQtcmQ',
 '41PtFgRt3DQa4J4FHDZkRA',
 'dcXeYPp3UAgiCzGzmEz6aQ',
 'h7skkHYh4JXZfARD6QhRBg',
 'WO10z5s4b6bg8G97emPn7A',
 'dZ800YCAc-HnvXInE7GU9g',
 'KR0O7xKEm6rgE711HkkXTA',
 'Choq0lds-DnnH0bUqx8FHQ',
 'ehbByKSGZ28AHXlqUCVzIA',
 'fBsbGcUFnMO_7yXfikNjGg',
 'iHdg7BupnCVc3FYlrzxTrg',
 'KYSd1eCFQY0uCON1GXmAhQ',
 'ILAaAXr4Varupm8MUSuwrA',
 '3B1z5h8Y39hnEXbRc9jTHA',
 'qjAgVbVXuECartJqPeXErQ',
 'u3f-W5yNwkSKJ8LjFtYCxg',
 'k7W_o3LpZaxMPWphXGS1MA',
 'WwNticOnmmkHrMRyAGR4rw',
 'H8_9t4Tlw2bcKW9qdqAM8w',
 'lBtiaHCwi-QHAs_ReWClhw',
 'AAZFsMBLNgNg58eA06bFJA',
 'yXgJ3_DPc4SQ-MDIw5Fkvw',
 '-n_qDyqJ6oq1yQb1qc0hsQ',
 'PIWXl9Bjw2Kvj6TUkIacsA']

In [28]:
#file_name = './Atlanta/Shortest Path/atlanta-shortest_path-zoning-FT_gpu.csv'
file_name = './Boston/Shortest Path/boston-shortest_path-zoning-FT_gpu.csv'

csv_file_02 = open(file_name, "w", newline='')
writer_path = csv.writer(csv_file_02, delimiter=',')
for result in results:
    if(len(result) > 0):
        writer_path.writerow(result)


## Execução em Paralelo

In [22]:
import multiprocessing as mp

from timeit import default_timer as timer
start = timer() 

print("Number of processors: ", mp.cpu_count())

# Step 1: Init multiprocessing.Pool()
#pool = mp.Pool(int(mp.cpu_count()))
pool = mp.Pool(int(2))

# Step 2: `pool.apply` the `howmany_within_range()`
items = [(object_id, poi_df) for object_id in obj_id_list]

results = pool.starmap(Shortest_Path_FT, items)

#Step 2.1 Writing results

#file_name = './geographic/SHORTEST PATH/austin-shortest_path-zoning-FT_gpu.csv'
file_name = './Atlanta/Shortest Path/atlanta-shortest_path-zoning-FT_gpu.csv'

csv_file_02 = open(file_name, "w", newline='')
writer_path = csv.writer(csv_file_02, delimiter=',')
for result in results:
    writer_path.writerow(result)

csv_file_02.close()

# Step 3: Don't forget to close
pool.close()
    
#print("without GPU:", timer()-start)
print("with GPU:", timer()-start) 

Number of processors:  20


2024-08-27 20:09:59,193| ERROR   | Exception (client): Error reading SSH protocol banner
2024-08-27 20:09:59,195| ERROR   | Traceback (most recent call last):
2024-08-27 20:09:59,196| ERROR   |   File "/home/salatiel/anaconda3/lib/python3.7/site-packages/paramiko/transport.py", line 2270, in _check_banner
2024-08-27 20:09:59,198| ERROR   |     buf = self.packetizer.readline(timeout)
2024-08-27 20:09:59,201| ERROR   |   File "/home/salatiel/anaconda3/lib/python3.7/site-packages/paramiko/packet.py", line 374, in readline
2024-08-27 20:09:59,202| ERROR   |     buf += self._read_timeout(timeout)
2024-08-27 20:09:59,202| ERROR   |   File "/home/salatiel/anaconda3/lib/python3.7/site-packages/paramiko/packet.py", line 603, in _read_timeout
2024-08-27 20:09:59,203| ERROR   |     raise EOFError()
2024-08-27 20:09:59,203| ERROR   | EOFError
2024-08-27 20:09:59,204| ERROR   | 
2024-08-27 20:09:59,204| ERROR   | During handling of the above exception, another exception occurred:
2024-08-27 20:09:5

Could not establish session to SSH gateway


2024-08-27 20:09:59,348| ERROR   | Exception (client): Error reading SSH protocol banner
2024-08-27 20:09:59,349| ERROR   | Traceback (most recent call last):
2024-08-27 20:09:59,350| ERROR   |   File "/home/salatiel/anaconda3/lib/python3.7/site-packages/paramiko/transport.py", line 2270, in _check_banner
2024-08-27 20:09:59,350| ERROR   |     buf = self.packetizer.readline(timeout)
2024-08-27 20:09:59,350| ERROR   |   File "/home/salatiel/anaconda3/lib/python3.7/site-packages/paramiko/packet.py", line 374, in readline
2024-08-27 20:09:59,351| ERROR   |     buf += self._read_timeout(timeout)
2024-08-27 20:09:59,351| ERROR   |   File "/home/salatiel/anaconda3/lib/python3.7/site-packages/paramiko/packet.py", line 603, in _read_timeout
2024-08-27 20:09:59,353| ERROR   |     raise EOFError()
2024-08-27 20:09:59,354| ERROR   | EOFError
2024-08-27 20:09:59,354| ERROR   | 
2024-08-27 20:09:59,357| ERROR   | During handling of the above exception, another exception occurred:
2024-08-27 20:09:5

Could not establish session to SSH gateway


...

2024-08-27 20:12:33,601| ERROR   | Exception (client): Error reading SSH protocol banner
2024-08-27 20:12:33,602| ERROR   | Traceback (most recent call last):
2024-08-27 20:12:33,603| ERROR   |   File "/home/salatiel/anaconda3/lib/python3.7/site-packages/paramiko/transport.py", line 2270, in _check_banner
2024-08-27 20:12:33,603| ERROR   |     buf = self.packetizer.readline(timeout)
2024-08-27 20:12:33,604| ERROR   |   File "/home/salatiel/anaconda3/lib/python3.7/site-packages/paramiko/packet.py", line 374, in readline
2024-08-27 20:12:33,605| ERROR   |     buf += self._read_timeout(timeout)
2024-08-27 20:12:33,605| ERROR   |   File "/home/salatiel/anaconda3/lib/python3.7/site-packages/paramiko/packet.py", line 603, in _read_timeout
2024-08-27 20:12:33,606| ERROR   |     raise EOFError()
2024-08-27 20:12:33,606| ERROR   | EOFError
2024-08-27 20:12:33,607| ERROR   | 
2024-08-27 20:12:33,607| ERROR   | During handling of the above exception, another exception occurred:
2024-08-27 20:12:3

Could not establish session to SSH gateway


2024-08-27 20:23:29,981| ERROR   | Exception (client): Error reading SSH protocol banner
2024-08-27 20:23:29,982| ERROR   | Traceback (most recent call last):
2024-08-27 20:23:29,982| ERROR   |   File "/home/salatiel/anaconda3/lib/python3.7/site-packages/paramiko/transport.py", line 2270, in _check_banner
2024-08-27 20:23:29,983| ERROR   |     buf = self.packetizer.readline(timeout)
2024-08-27 20:23:29,983| ERROR   |   File "/home/salatiel/anaconda3/lib/python3.7/site-packages/paramiko/packet.py", line 374, in readline
2024-08-27 20:23:29,983| ERROR   |     buf += self._read_timeout(timeout)
2024-08-27 20:23:29,983| ERROR   |   File "/home/salatiel/anaconda3/lib/python3.7/site-packages/paramiko/packet.py", line 603, in _read_timeout
2024-08-27 20:23:29,983| ERROR   |     raise EOFError()
2024-08-27 20:23:29,984| ERROR   | EOFError
2024-08-27 20:23:29,984| ERROR   | 
2024-08-27 20:23:29,984| ERROR   | During handling of the above exception, another exception occurred:
2024-08-27 20:23:2

Could not establish session to SSH gateway


2024-08-27 20:43:26,796| ERROR   | Could not establish connection from local ('127.0.0.1', 38387) to remote ('localhost', 5432) side of the tunnel: 


In [49]:
print(len(results))

105


In [22]:
#file_name = './geographic/SHORTEST PATH/austin-sl-corpus-' + str(bin_number) + '-c_.csv'
file_name = './geographic/SHORTEST PATH/austin-shortest_path-' + str(bin_number) + '-CF-c_.csv'
#file_name = './geographic/SHORTEST PATH/austin-shortest_path-' + str(bin_number) + '-FT-c.csv'

csv_file_02 = open(file_name, "w", newline='')
writer_path = csv.writer(csv_file_02, delimiter=',')
for result in results:
    writer_path.writerow(result)

In [22]:
print('ok')

ok
